In [1]:
from ML_Preparation.Preprocessing_M import DataExtractor, DataPreprocessing

de = DataExtractor("D:/hka-aqm-m", "C:/Users/danie/building_M_data")

df = de.create_df()

No .dat files found in C:/Users/danie/building_M_data. 
 Trying to extract files from the original directory D:/hka-aqm-m
Read data successfully.
Data contains 979166 data points and 18 columns.


In [2]:
dp = DataPreprocessing()

df_preprocessed = dp.preprocess_df(df, rolling_window = "7d", sample_time = "1d")

Number of outliers detected: -73438
Number of normal samples detected: 905728


In [3]:
df_preprocessed.columns

Index(['date_time', 'tmp', 'hum', 'CO2', 'VOC', 'vis', 'IR', 'BLE', 'rssi',
       'snr', 'tavg', 'tmin', 'tmax', 'prcp', 'wdir', 'wspd', 'wpgt', 'pres',
       'room_number', 'time_diff_sec', 'hum_diff', 'CO2_diff', 'VOC_diff',
       'vis_diff', 'IR_diff', 'BLE_diff', 'tavg_diff', 'tmin_diff',
       'tmax_diff', 'prcp_diff', 'wdir_diff', 'wspd_diff', 'wpgt_diff',
       'pres_diff', 'year', 'dayofweek', 'hour', 'season', 'color',
       'VOC_CO2_ratio'],
      dtype='object')

In [4]:
# df_preprocessed.to_parquet("hka_M_preprocessed.parquet")

In [5]:
# import pandas as pd
# df_preprocessed = pd.read_parquet("hka_M_preprocessed.parquet")

In [6]:
# from Feature_Engineering import *
# fe = FeatureEngineering(df_preprocessed.drop(columns = ["VOC_diff", "VOC_diff_per_sec", "VOC_CO2_ratio"]), label = "CO2", categorical_features = ["season", "room_number", "dayofweek", "color"])

In [7]:
from ML_Preparation.Feature_Engineering import *
fe = FeatureEngineering(df_preprocessed.drop(columns = ["VOC_CO2_ratio"]), label = "CO2", categorical_features = ["season", "room_number", "dayofweek", "color"])

In [8]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(fe.X_train, fe.y_train)

LinearRegression()

In [9]:
from sklearn.metrics import r2_score, mean_absolute_percentage_error

pred_test = lr.predict(fe.X_test)

pred_train = lr.predict(fe.X_train)

print(f"Training score: {mean_absolute_percentage_error(y_true = fe.y_train, y_pred = pred_train)}")

print(f"Training score: {mean_absolute_percentage_error(y_true = fe.y_test, y_pred = pred_test)}")

Training score: 0.12344784104587163
Training score: 12813291765.391193


In [10]:
fe.X_train.shape

(5671, 71)

In [11]:
time_steps = 7
def create_ds(data, t_steps):
    data = pd.DataFrame(data)
    data_s = data.copy()
    for i in range(time_steps):
        data = pd.concat([data, data_s.shift(-(i+1))], axis = 1)   
    data.dropna(axis=0, inplace=True)
    return data.values

X_train_stepped = create_ds(fe.X_train, 7)

In [12]:
y_train_stepped = create_ds(fe.y_train, 7)

In [13]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU, Dropout, Bidirectional
from keras.optimizers import Adam, SGD

class MyLSTMModel:
    def __init__(self, lstm_units=int(fe.X_train.shape[1]),input_shape=(None, fe.X_train.shape[1])):
        self.model = Sequential()
        self.model.add(LSTM(units=lstm_units, input_shape=input_shape, return_sequences = True, recurrent_dropout = 0.3))
        self.model.add(Dropout(0.3))
        self.model.add(LSTM(units=lstm_units*2, return_sequences = True, recurrent_dropout = 0.3))
        self.model.add(Dropout(0.3))
        self.model.add(Bidirectional(LSTM(units=lstm_units, recurrent_dropout = 0.3)))
        self.model.add(Dense(60, activation = "relu"))
        self.model.add(Dense(40, activation = "relu"))
        self.model.add(Dense(20, activation = "relu"))

        self.model.add(Dense(1))

        self.model.compile(optimizer=Adam(learning_rate = 0.05), loss='mse')
    
    def summary(self):
        self.model.summary()

# Instantiate the model
my_model = MyLSTMModel()

# Print the model summary
X_train_array = fe.X_train.values.reshape(fe.X_train.shape[0], 1, fe.X_train.shape[1]).astype(np.float32)
y_train_array = fe.y_train.values.astype(np.float32).reshape(fe.y_train.shape[0], 1, 1)
my_model.model.fit(X_train_array, y_train_array, epochs = 15, batch_size = 200)

Epoch 1/15


c:\Users\danie\anaconda3\envs\awp1\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 342040.0312
Epoch 2/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 56885.1211 
Epoch 3/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 15453.2451
Epoch 4/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 11138.4834
Epoch 5/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 9034.4854
Epoch 6/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7899.9814
Epoch 7/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6840.0557
Epoch 8/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5785.7495
Epoch 9/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5626.1987
Epoch 10/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5550.1978
Epoch 11/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4362.8916
Epoch 12/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4722.0991
Epoch 13/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4061.9287
Epoch 14/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4019.4763
Epoch 15/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/st

In [14]:
X_test_array = fe.X_test.values.reshape(fe.X_test.shape[0], 1, fe.X_test.shape[1]).astype(np.float32)
y_test_array = fe.y_test.values.astype(np.float32).reshape(fe.y_test.shape[0], 1, 1)
pred = my_model.model.predict(X_test_array)

45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step 


In [15]:
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_absolute_error, mean_squared_error

def evaluate_model(y_true, y_pred):
    eval_df = dict()

    eval_df["r2"] = r2_score(y_pred = y_pred, y_true = y_true).round(2)
    eval_df["mape"] = mean_absolute_percentage_error(y_pred = y_pred, y_true = y_true).round(2)
    eval_df["mae"] = mean_absolute_error(y_pred = y_pred, y_true = y_true).round(2)
    eval_df["mse"] = mean_squared_error(y_pred = y_pred, y_true = y_true).round(2)

    return pd.DataFrame([eval_df])

evaluate_model(fe.y_test, pred)


,r2,mape,mae,mse
0,-0.88,0.1,51.97,8059.51


In [16]:
evaluate_model(fe.y_train, my_model.model.predict(X_train_array))

178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 596us/step


,r2,mape,mae,mse
0,0.9,0.07,41.45,2906.82


In [17]:
test_df = fe.df.loc[fe.y_test.index, :]

In [18]:
import plotly.express as px

px.scatter(x = fe.y_test.index, y = [fe.y_test, pd.DataFrame(pred)[0]])

In [20]:
fe.y_test

date_time
2023-06-28    450.385806
2023-06-28    445.681452
2023-06-28    592.311033
2023-06-28    470.180484
2023-06-28    553.289269
                 ...    
2023-09-26    449.402520
2023-09-26    550.050000
2023-09-26    500.018858
2023-09-26    561.655217
2023-09-26    504.308540
Name: CO2, Length: 1417, dtype: float64

In [33]:
y_values = pd.DataFrame(fe.y_test).iloc[:-1]
y_values["CO2_pred"] = pred

In [66]:
fe.X_test.columns

Index(['year', 'hour', 'tmp', 'hum', 'VOC', 'vis', 'IR', 'BLE', 'tavg', 'tmin',
       'tmax', 'prcp', 'wdir', 'wspd', 'wpgt', 'pres', 'hum_diff', 'CO2_diff',
       'VOC_diff', 'vis_diff', 'IR_diff', 'BLE_diff', 'tavg_diff', 'tmin_diff',
       'tmax_diff', 'prcp_diff', 'wdir_diff', 'wspd_diff', 'wpgt_diff',
       'pres_diff', 'season_fall', 'season_spring', 'season_summer',
       'season_winter', 'room_number_m001', 'room_number_m002',
       'room_number_m003', 'room_number_m004', 'room_number_m101',
       'room_number_m102', 'room_number_m103', 'room_number_m104',
       'room_number_m105', 'room_number_m113', 'room_number_m201',
       'room_number_m202', 'room_number_m206', 'room_number_m207',
       'room_number_m209', 'room_number_m216', 'room_number_m301',
       'room_number_m302', 'room_number_m303', 'room_number_m304',
       'room_number_m306', 'room_number_m310', 'room_number_m700',
       'room_number_mu04', 'room_number_mu06', 'room_number_mu10',
       'room_number_

In [86]:
mapping = dict()

for col in fe.X_test.columns:
   if "room_" in col:
      mapping[col] = col.split("_")[-1]

In [78]:
mapping

{'room_number_m001': 'm001',
 'room_number_m002': 'm002',
 'room_number_m003': 'm003',
 'room_number_m004': 'm004',
 'room_number_m101': 'm101',
 'room_number_m102': 'm102',
 'room_number_m103': 'm103',
 'room_number_m104': 'm104',
 'room_number_m105': 'm105',
 'room_number_m113': 'm113',
 'room_number_m201': 'm201',
 'room_number_m202': 'm202',
 'room_number_m206': 'm206',
 'room_number_m207': 'm207',
 'room_number_m209': 'm209',
 'room_number_m216': 'm216',
 'room_number_m301': 'm301',
 'room_number_m302': 'm302',
 'room_number_m303': 'm303',
 'room_number_m304': 'm304',
 'room_number_m306': 'm306',
 'room_number_m310': 'm310',
 'room_number_m700': 'm700',
 'room_number_mu04': 'mu04',
 'room_number_mu06': 'mu06',
 'room_number_mu10': 'mu10',
 'room_number_mu11': 'mu11'}

In [81]:
fe.X_test.idxmax(axis=1).map(mapping)

date_time
2023-06-28    m304
2023-06-28    m101
2023-06-28    m002
2023-06-28    m003
2023-06-28    mu06
              ... 
2023-09-26    m101
2023-09-26    m201
2023-09-26    mu11
2023-09-26    m302
2023-09-26    mu04
Length: 1417, dtype: object